In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [9]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train_examples = mnist.train.num_examples
validation_examples = mnist.validation.num_examples
test_examples = mnist.test.num_examples
print(train_examples,validation_examples,test_examples)

batch_size =100
xs,ys = mnist.train.next_batch(batch_size)
print('X shape:',xs.shape)
print('Y shape:',ys.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 5000 10000
X shape: (100, 784)
Y shape: (100, 10)


In [3]:
#tensorflow训练神经网络
import tensorflow as tf

#mnist 数据集的相关常数
INPUT_NODE = 784
OUTPUT_NODE = 10
# 配置神经网络的参数
LAYER1_NODE = 500
BATCH_SIZE =100
LEARNING_RATE_BASE =0.8
LEARNING_RATE_DECAY =0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS =3000
MOVING_AVERAGE_DECAY = 0.99

# 一个辅助函数，给定神经网络的输入和所有参数，计算神经网络的前向传播结果
def inference(input_tensor,avg_class,weight1,biases1,weight2,biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weight1)+biases1)
        return tf.matmul(layer1,weight2)+biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weight1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weight2))+avg_class.average(biases2)
    
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name ='y-input')
    
    #生成隐藏层的参数
    weihgt1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层的参数
    weight2 =tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 =tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算在当前参数下神经网络前向传播的结果，这里给出的计算滑动平均类为None,所以含少数不会使用参数的滑动平均值
    y=inference(x,None,weihgt1,biases1,weight2,biases2)
    
#     定义存储训练轮数的变量，这个变量不需要计算滑动平均值，，一般讲代表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0,trainable=False)
    
#     给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类，给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages= tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
#     在所有代表神经网络参数的变量上使用滑动平均，其他辅助变量就不需要了
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    ###########
    #计算使用了滑动平均之后的前向传播结果，滑动平均不会改变变量本身的值，而是会维护一个影子变量来记录其滑动平均值
    average_y = inference(x,variable_averages,weihgt1,biases1,weight2,biases2)
    #计算交叉熵作为刻画预测值与真实值之间差距的损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2 正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
#     -计算模型的正则化损失，一般只使用权重的正则化损失，而不使用偏置项
    regularization = regularizer(weihgt1)+regularizer(weight2)
#     总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
#     设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples,LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    #     在训练神经网络时，每更新一遍数据，就会通过反向传播更新擦书，又要更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op =tf.no_op(name ='train')

    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #初始化会话，开始训练
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}

        test_feed = {x:mnist.test.images,y_:mnist.test.labels}

        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print('After %d training steps,validation accuracy using average model is %g'%(i,validate_acc))

            xs,ys = mnist.train.next_batch(BATCH_SIZE)  
            sess.run(train_op,feed_dict={x:xs,y_:ys})
            
    #     在训练之后，在测试集上检验神经网络模型的最终正确率
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print('After %d training steps,test accuracy using average model is %g'%(TRAINING_STEPS,test_acc))
    
    
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)
    train(mnist)
    
if __name__ =='__main__':
    tf.app.run()
        
    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training steps,validation accuracy using average model is 0.1108
After 1000 training steps,validation accuracy using average model is 0.9774
After 2000 training steps,validation accuracy using average model is 0.9832
After 3000 training steps,test accuracy using average model is 0.9821


SystemExit: 

d:\python3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
# tensorflow模型持久化 saver
# 保存模型
import tensorflow as tf

#声明两个变量并计算它们的和
v1 = tf.Variable(tf.constant(1.0,shape=[1]),dtype=tf.float32,name='v1')
v2 = tf.Variable(tf.constant(2.0,shape=[1]),dtype=tf.float32,name='v2')
result =v1 + v2

init_op =tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
#     将模型保存起来
    saver.save(sess,"chi_jiu_hua/model.ckpt")
tf.reset_default_graph()


In [20]:
# 加载模型  不需要对变量进行初始化
import tensorflow as tf
v1 =tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2 =tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result = v1+v2
saver = tf.train.Saver()
with tf.Session() as sess:    
#     加载已保存的模型
    saver.restore(sess,"chi_jiu_hua/model.ckpt")
    print(sess.run(result))


INFO:tensorflow:Restoring parameters from chi_jiu_hua/model.ckpt
[ 3.]


In [11]:
import tensorflow as tf
saver =tf.train.import_meta_graph('chi_jiu_hua/model.ckpt.meta')

with tf.Session() as sess:
    saver.restore(sess,'chi_jiu_hua/model.ckpt')
#     通过张量的名称来获取张量 必须是张量

    print(sess.run(tf.get_default_graph().get_tensor_by_name('add:0')))

INFO:tensorflow:Restoring parameters from chi_jiu_hua/model.ckpt
[ 3.]


默认保存和加载了tensorflow计算图上定义的全部变量，但有时只需要保存活着加载部分变量，
比如之前训练好了五层，现在想加入第六层，可以将前五层直接加入到新的模型，仅仅将最后一层的神经网络进行训练

In [17]:
saver = tf.train.Saver([v1])
# 只有变量v1会被加载进来
print(saver)
tf.reset_default_graph()

In [12]:
# 变量重命名如何被使用  这样做的方法主要是方便使用变量的滑动平均值
v1 = tf.Variable(tf.constant(1.0,shape=[1]),name = 'other-v1')
v2 = tf.Variable(tf.constant(2.0,shape=[1]),name = 'other-v1')
saver =tf.train.Saver()
with tf.Session() as sess:
#     此时加载会报错、 变量找不到
#     saver.restore(sess,'chi_jiu_hua/model.ckpt')

#使用一个字典来重命名变量就可以加载原来的模型了。这个字典制定了原来名称为v1的变量现在加载到v1中，（名称为other-v1）
    saver = tf.train.Saver({'v1':v1,'v2':v2})
    saver.restore(sess,'chi_jiu_hua/model.ckpt')
    print(v1.eval())


INFO:tensorflow:Restoring parameters from chi_jiu_hua/model.ckpt
[ 1.]


In [27]:
# 滑动平均模型
import tensorflow as tf
v = tf.Variable(0,dtype=tf.float32,name = 'v')
for variables in tf.global_variables():
    print(variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
# 在申明滑动平均模型之后，tensorflow会自动生成一个影子变量
for variables in tf.global_variables():
    print(variables.name)
    
saver =tf.train.Saver()
with tf.Session() as sess:
    init_op =tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v,10))
    saver.save(sess,'Hua_Dong_Ping_Jun/model.ckpt')
    print(sess.run([v,ema.average(v)]))
tf.reset_default_graph()

v1:0
v2:0
other-v1:0
other-v2:0
other-v1_1:0
other-v2_1:0
other-v1_2:0
other-v2_2:0
v:0
v1/ExponentialMovingAverage:0
v2/ExponentialMovingAverage:0
other-v1/ExponentialMovingAverage:0
other-v2/ExponentialMovingAverage:0
other-v1_1/ExponentialMovingAverage:0
other-v2_1/ExponentialMovingAverage:0
other-v1_2/ExponentialMovingAverage:0
other-v2_2/ExponentialMovingAverage:0
v/ExponentialMovingAverage:0
v_1:0
v1/ExponentialMovingAverage_1:0
v2/ExponentialMovingAverage_1:0
other-v1/ExponentialMovingAverage_1:0
other-v2/ExponentialMovingAverage_1:0
other-v1_1/ExponentialMovingAverage_1:0
other-v2_1/ExponentialMovingAverage_1:0
other-v1_2/ExponentialMovingAverage_1:0
other-v2_2/ExponentialMovingAverage_1:0
v/ExponentialMovingAverage_1:0
v1/ExponentialMovingAverage/ExponentialMovingAverage:0
v2/ExponentialMovingAverage/ExponentialMovingAverage:0
other-v1/ExponentialMovingAverage/ExponentialMovingAverage:0
other-v2/ExponentialMovingAverage/ExponentialMovingAverage:0
other-v1_1/ExponentialMovingAv

In [30]:
# 通过变量重命名直接读取变量的滑动均值
v = tf.Variable(0,dtype=tf.float32,name='v')
saver = tf.train.Saver({'v/ExponentialMovingAverage':v})
with tf.Session() as sess:
    saver.restore(sess,'Hua_Dong_Ping_Jun/model.ckpt')
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from Hua_Dong_Ping_Jun/model.ckpt
0.0


In [32]:
# 为了方便加载时重命名滑动平均变量，使用variables_to_restore

import tensorflow as tf
v =tf.Variable(0,dtype=tf.float32,name='v')
ema= tf.train.ExponentialMovingAverage(0.99)

# 通过使用variables_to_restore函数可以直接生成上面代码中提供的字典
print(ema.variables_to_restore())

saver =tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,'Hua_Dong_Ping_Jun/model.ckpt')
    print(sess.run(v))


{'v_2/ExponentialMovingAverage': <tf.Variable 'v_2:0' shape=() dtype=float32_ref>, 'v_1/ExponentialMovingAverage': <tf.Variable 'v_1:0' shape=() dtype=float32_ref>, 'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>, 'v_3/ExponentialMovingAverage': <tf.Variable 'v_3:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from Hua_Dong_Ping_Jun/model.ckpt
10.0


In [ ]:

# 将计算图中的变量及取值通过常量的方式保存
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2 = tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result =v1 + v2

init_op =tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
#     导出当前层得计算过程，只需要这部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess,graph_def,['add'])
#     将导出的模型存入到文件
    with tf.gfile.GFile('save_variables_as_constants/model.pb','wb') as f:
        f.write(output_graph_def.SerializeToString())
    

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename = 'save_variables_as_constants/model.pb'
    with gfile.FastGFile(model_filename,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
#     将graph_def中保存的图片加载到当前的图中
    result = tf.import_graph_def(graph_def,return_elements=['add:0'])
    print(sess.run(result))
        

ERROR! Session/line number was not unique in database. History logging moved to new session 1043


In [ ]:

from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次100张照片
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784],name='x-input')
y = tf.placeholder(tf.float32,[None,10])

#创建一个简单的神经网络，输入层784个神经元，输出层10个神经元
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x,W)+b, name='output')

#二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})

        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))
    #保存模型参数和结构
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['output'])
    # 保存模型到目录下的model文件夹中
    with tf.gfile.FastGFile('./models/tfmodel.pb',mode='wb') as f:
        f.write(output_graph_def.SerializeToString())

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter 0,Testing Accuracy 0.824
Iter 1,Testing Accuracy 0.8825
Iter 2,Testing Accuracy 0.8997
Iter 3,Testing Accuracy 0.9046
Iter 4,Testing Accuracy 0.9082
Iter 5,Testing Accuracy 0.9099
Iter 6,Testing Accuracy 0.9116
Iter 7,Testing Accuracy 0.9128
Iter 8,Testing Accuracy 0.9136
Iter 9,Testing Accuracy 0.9152
Iter 10,Testing Accuracy 0.9179
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [5]:
#保存为json格式
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2 = tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result =v1 + v2
saver = tf.train.Saver()
saver.export_meta_graph('Dataformat_Json/model.ckpt.meda.json',as_text=True)

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_HALF
            type: DT_BFLOAT16
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "ApplyGradientDescent"
      input_arg {
        name: "var"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "alpha"
        type_attr: "T"
      }
      input_arg {
        name: "delta"
        type_attr: "T"


In [ ]:
# 查看checkpoint保存的变量
import tensorflow as tf
reader = tf.train.NewCheckpointReader('Wacth_Checkpoint/model.ckpt')
# 获取所有变量的列表
global_variables = reader.get_variable_to_shape_map()
for variable_name in global_variables:
    print(variable_name,global_variables[variable_name])
    
# 获取名称为v1的变量的取值
print('Value for variable v1 is:',reader.get_tensor('v1'))


In [20]:
#指数衰减+滑动平均+梯度下降
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
BATCH_SIZE = 10
LEARNING_RATE_BASE = 0.8
LEARNING_RRATE_DECAY = 0.99
REGULARRIZATON_RATE = 0.0001
TRAINING_STEEPS =300
MOVING_AVERAGE_DECAY = 0.99

# 模型保存的路径和文件名
MODEL_SAVE_PATH ='Mnistdata_Model'
MODEL_NAME ='model.ckpt'

# 定义神经网络相关的参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER_NODE = 500

def get_weight_variable(shape,regularizer):
    weights = tf.get_variable("weights",shape,initializer = tf.truncated_normal_initializer(stddev=0.1))
    #当给出了正则化生成函数时，将当前变量的正则化损失加入名字为losses的集合中
    if regularizer != None:
        tf.add_to_collection('losses',regularizer(weights))
    return weights

# 定义神经网络向前传播的过程
def inference(input_tensor,regularizer):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE,LAYER_NODE],regularizer)
        biases = tf.get_variable('biases',[LAYER_NODE],initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
    
# 类似的声明第二层神经网络的变量并完成前向传播的过程
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER_NODE,OUTPUT_NODE],regularizer)
        biases = tf.get_variable('biases',[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
    layer2 =tf.nn.relu(tf.matmul(layer1,weights)+biases)
    return layer2

def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name ='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARRIZATON_RATE)
    y = inference(x,regularizer)
    
    global_step = tf.Variable(0,trainable=False)

#     global_step = tf.Variable(0)
    variable_average = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op = variable_average.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RRATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
#             print(xs,'\n',ys)
            _,loss_value,step = sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            
            if i%1000 ==0:
#                 print('After %d training step(s),loss on training batch is %g'%(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)
#     tf.reset_default_graph()          
                
def main(argv=None):
    mnist =input_data.read_data_sets('MNIST_data/',one_hot=True)
    train(mnist)
    
if __name__ == '__main__': 
    tf.reset_default_graph()
    tf.app.run()
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


SystemExit: 

d:\python3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# 用存储的模型，用验证集测试模型
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import time

BATCH_SIZE = 10
LEARNING_RATE_BASE = 0.8
LEARNING_RRATE_DECAY = 0.99
REGULARRIZATON_RATE = 0.0001
TRAINING_STEEPS =300
MOVING_AVERAGE_DECAY = 0.99

# 模型保存的路径和文件名
MODEL_SAVE_PATH ='Mnistdata_Model'
MODEL_NAME ='model.ckpt'

# 定义神经网络相关的参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER_NODE = 500


EVAL_INTERVAL_SECS=10
def get_weight_variable(shape,regularizer):
    weights = tf.get_variable("weights",shape,initializer = tf.truncated_normal_initializer(stddev=0.1))
    #当给出了正则化生成函数时，将当前变量的正则化损失加入名字为losses的集合中
    if regularizer != None:
        tf.add_to_collection('losses',regularizer(weights))
    return weights

# 定义神经网络向前传播的过程
def inference(input_tensor,regularizer):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE,LAYER_NODE],regularizer)
        biases = tf.get_variable('biases',[LAYER_NODE],initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
    
    # 类似的声明第二层神经网络的变量并完成前向传播的过程
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER_NODE,OUTPUT_NODE],regularizer)
        biases = tf.get_variable('biases',[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
    layer2 =tf.nn.relu(tf.matmul(layer1,weights)+biases)
    return layer2
        
# 每10秒加载一次最新的模型，并在测试
EVAL_INTERVAL_SECS = 10
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
        
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        #前向传播
        regularizer = tf.contrib.layers.l2_regularizer(REGULARRIZATON_RATE)
        y = inference(x,None)
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
       
        global_step = tf.Variable(0,trainable=False)
        variable_average = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore=variable_average.variables_to_restore()
        
        saver = tf.train.Saver(variables_to_restore)      
        
#         每隔EVAL_INTERVAL_SECS秒钟秒调用一次计算正确率的过程以检测训练过程钟正确率的变化
        while True:
            with tf.Session() as sess:
                ckpt=tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
#                     加载模型
                    saver.restore(sess,ckpt.model_checkpoint_path)
#                     通过文件名得到模型保存时迭代的轮数
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score=sess.run(accuracy,feed_dict=validate_feed)
                    print('After %s training steps,validation accracy=%g'%(global_step,accuracy_score))
                else:
                    print('No checkpoint file found.')
                    return
            time.sleep(EVAL_INTERVAL_SECS)

def main():
    mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
    evaluate(mnist)
    
if __name__ == '__main__':
    main()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 trainin

INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters fro

After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accrac

INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters fro

After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accrac

...

INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters fro

After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accrac

INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters fro

After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accrac

INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters from Mnistdata_Model\model.ckpt-1
After 1 training steps,validation accracy=0.1012
INFO:tensorflow:Restoring parameters fro

In [3]:
# 保存参数  达到某个Loss值就开始保存参数
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()
# 定义隐藏层输入输出层的节点数
INPUT_NODE=784
OUTPUT_NODE=10
LEARNING_RATE=0.01
DECAY =0.8
BATCH_SIZE =8

x=tf.placeholder(tf.float32,shape=[None,INPUT_NODE],name='x_input')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT_NODE],name='x_input')
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

def init_weight(x,shapes,regularization):
    weights = tf.Variable(tf.truncated_normal(shapes,stddev=0.01),name='weights')
    biases = tf.Variable(tf.constant(0.01,shape=[shapes[1]]),dtype=tf.float32,name='biases')
    # 如果L2正则化存在的话，就加入L2正则化，反正到最后都是要进行relu激活函数
    if regularization != None:
        tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularization)(weights))
    return tf.nn.relu(tf.matmul(x,weights)+biases)
    
y = init_weight(x,[INPUT_NODE,OUTPUT_NODE],None)
global_step = tf.Variable(0,trainable=False)

# 设定指数衰减学习率
learning_rate = tf.train.exponential_decay(0.1,global_step,mnist.train.num_examples/BATCH_SIZE,DECAY)

#设定滑动平均模型
ema =tf.train.ExponentialMovingAverage(DECAY,global_step)
maintain_average_step=ema.apply(tf.trainable_variables())

# 定义交叉熵损失函数
cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
# 所有样例的交叉熵平均值
cross_entropy_mean = tf.reduce_mean(cross_entropy)
# 将交叉熵损失函数加入losses集合
tf.add_to_collection('losses',cross_entropy_mean)
loss =tf.add_n(tf.get_collection('losses'))

#使用梯度下降来优化损失函数 不懂global_step的作用
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
# 在训练神经网络模型时，每过一遍数据就需要通过反向传播来更新网络中的参数，又要更新每一个参数的滑动平均值
with tf.control_dependencies([train_step,maintain_average_step]):
    train_op = tf.no_op(name='train')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        xs,ys = mnist.train.next_batch(BATCH_SIZE)
#         print(xs.shape,'\n',ys.shape)
#         train_step,loss,global_step=sess.run([train_step,loss,global_step],feed_dict={x:xs,y_:ys})
        
        losses=sess.run(loss,feed_dict={x:xs,y_:ys})
        if losses < 2.2:
            print(losses)
            break
        
#         如果满足条件就跳出循环
        saver =tf.train.Saver()
        saver.save(sess,'mnist_learningrate/model.ckpt')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# 读取保存的参数
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()
# 定义隐藏层输入输出层的节点数
INPUT_NODE=784
OUTPUT_NODE=10
LEARNING_RATE=0.01
DECAY =0.8
BATCH_SIZE =8

x=tf.placeholder(tf.float32,shape=[None,INPUT_NODE],name='x_input')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT_NODE],name='x_input')
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

def init_weight(x,shapes,regularization):
    weights = tf.Variable(tf.truncated_normal(shapes,stddev=0.01),name='weights')
    biases = tf.Variable(tf.constant(0.01,shape=[shapes[1]]),dtype=tf.float32,name='biases')
    # 如果L2正则化存在的话，就加入L2正则化，反正到最后都是要进行relu激活函数
    if regularization != None:
        tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularization)(weights))
    return tf.nn.relu(tf.matmul(x,weights)+biases)
    
y = init_weight(x,[INPUT_NODE,OUTPUT_NODE],None)
global_step = tf.Variable(0,trainable=False)

# 设定指数衰减学习率
learning_rate = tf.train.exponential_decay(0.1,global_step,mnist.train.num_examples/BATCH_SIZE,DECAY)

#设定滑动平均模型
ema =tf.train.ExponentialMovingAverage(DECAY,global_step)
maintain_average_step=ema.apply(tf.trainable_variables())

# 定义交叉熵损失函数
cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
# 所有样例的交叉熵平均值
cross_entropy_mean = tf.reduce_mean(cross_entropy)
# 将交叉熵损失函数加入losses集合
tf.add_to_collection('losses',cross_entropy_mean)
loss =tf.add_n(tf.get_collection('losses'))

#使用梯度下降来优化损失函数 不懂global_step的作用
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
# 在训练神经网络模型时，每过一遍数据就需要通过反向传播来更新网络中的参数，又要更新每一个参数的滑动平均值
with tf.control_dependencies([train_step,maintain_average_step]):
    train_op = tf.no_op(name='train')

with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
    for i in range(300):
        xs,ys = mnist.train.next_batch(BATCH_SIZE)
        saver =tf.train.Saver()
        saver.restore(sess,'mnist_learningrate/model.ckpt')
        print(losses)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:t

INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters fro

2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parameters from mnist_learningrate/model.ckpt
2.27839
INFO:tensorflow:Restoring parame

In [10]:
#tensorflow训练神经网络
#L2正则化防止过拟合，滑动平均  指数衰减   梯度下降
import tensorflow as tf

#mnist 数据集的相关常数
INPUT_NODE = 784
OUTPUT_NODE = 10
# 配置神经网络的参数
LAYER1_NODE = 500
BATCH_SIZE =100
LEARNING_RATE_BASE =0.8
LEARNING_RATE_DECAY =0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS =300
MOVING_AVERAGE_DECAY = 0.99

# 一个辅助函数，给定神经网络的输入和所有参数，计算神经网络的前向传播结果
def inference(input_tensor,avg_class,weight1,biases1,weight2,biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weight1)+biases1)
        return tf.matmul(layer1,weight2)+biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weight1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weight2))+avg_class.average(biases2)
    
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name ='y-input')
    
    #生成隐藏层的参数
    weihgt1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层的参数
    weight2 =tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 =tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算在当前参数下神经网络前向传播的结果，这里给出的计算滑动平均类为None,所以参数不会使用参数的滑动平均值
    y=inference(x,None,weihgt1,biases1,weight2,biases2)
    
#     定义存储训练轮数的变量，这个变量不需要计算滑动平均值，，一般讲代表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0,trainable=False)
    
#     给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类，给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages= tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
#     在所有代表神经网络参数的变量上使用滑动平均，其他辅助变量就不需要了
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    ###########
    #计算使用了滑动平均之后的前向传播结果，滑动平均不会改变变量本身的值，而是会维护一个影子变量来记录其滑动平均值
    average_y = inference(x,variable_averages,weihgt1,biases1,weight2,biases2)
    #计算交叉熵作为刻画预测值与真实值之间差距的损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2 正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
#     -计算模型的正则化损失，一般只使用权重的正则化损失，而不使用偏置项
    regularization = regularizer(weihgt1)+regularizer(weight2)
#     总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
#     设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples,LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    #     在训练神经网络时，每更新一遍数据，就会通过反向传播更新参数，又要更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op =tf.no_op(name ='train')

    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #初始化会话，开始训练
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}

        test_feed = {x:mnist.test.images,y_:mnist.test.labels}

        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print('After %d training steps,validation accuracy using average model is %g'%(i,validate_acc))

            xs,ys = mnist.train.next_batch(BATCH_SIZE)  
            sess.run(train_op,feed_dict={x:xs,y_:ys})
            
    #     在训练之后，在测试集上检验神经网络模型的最终正确率
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print('After %d training steps,test accuracy using average model is %g'%(TRAINING_STEPS,test_acc))
    
    
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)
    train(mnist)
    
if __name__ =='__main__':
    tf.app.run()
        
    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training steps,validation accuracy using average model is 0.1448
After 300 training steps,test accuracy using average model is 0.9593


SystemExit: 

d:\python3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
